# SDG 13 – NYC Monthly CO₂ Forecast

**Goal**: Predict city‑level CO₂ emissions to support climate‑action policies.

**Model**: XGBoost Regressor with lagged & seasonal features.

---

## 1. Synthetic Data Generation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import warnings, joblib
warnings.filterwarnings('ignore')

sns.set(style='whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# 2018‑01 → 2024‑12 (84 months)
np.random.seed(42)
dates = pd.date_range('2018-01-01', '2024-12-31', freq='MS')
n = len(dates)

data = pd.DataFrame({
    'date': dates,
    'month': dates.month,
    'year': dates.year,
    'temperature_c': 10 + 8*np.sin(2*np.pi*(dates.month-1)/12) + np.random.normal(0,2,n),
    'energy_consumption_gwh': 3000 + 800*np.abs(np.sin(2*np.pi*(dates.month-1)/12)) + np.random.normal(0,200,n) + 20*(dates.year-2018)
})

base = 1200 + 0.25*data['energy_consumption_gwh']
heating = np.maximum(10 - data['temperature_c'],0)*60
cooling = np.maximum(data['temperature_c']-25,0)*50
trend = 15*(data['year']-2018)
noise = np.random.normal(0,50,n)

data['co2_kt'] = base + heating + cooling + trend + noise
data['co2_kt'] = data['co2_kt'].clip(lower=1000)
data['is_holiday_month'] = data['month'].isin([1,12]).astype(int)

data = data[['date','temperature_c','energy_consumption_gwh','co2_kt','month','year','is_holiday_month']]
data.head()

## 2. EDA

In [ ]:
plt.plot(data['date'], data['co2_kt'], label='CO₂ (kt)', color='darkred')
plt.title('Monthly CO₂ Emissions')
plt.legend(); plt.show()

sns.scatterplot(x='temperature_c', y='co2_kt', hue='month', data=data, palette='coolwarm')
plt.title('CO₂ vs Temperature'); plt.show()

corr = data[['co2_kt','temperature_c','energy_consumption_gwh']].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm'); plt.show()

## 3. Feature Engineering

In [ ]:
for lag in [1,2,3]:
    data[f'co2_lag_{lag}'] = data['co2_kt'].shift(lag)
data['co2_roll_mean_3'] = data['co2_kt'].shift(1).rolling(3).mean()
data['co2_roll_std_3']  = data['co2_kt'].shift(1).rolling(3).std()

data = pd.get_dummies(data, columns=['month'], prefix='month', drop_first=True)
data = data.dropna().reset_index(drop=True)
data.head()

## 4. Train‑Test Split

In [ ]:
feature_cols = [c for c in data.columns if c not in ['date','co2_kt']]
X = data[feature_cols]
y = data['co2_kt']

train_size = len(data) - 12
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

print(f"Train: {len(X_train)}  Test: {len(X_test)}")

## 5. XGBoost Training

In [ ]:
tscv = TimeSeriesSplit(n_splits=5)
param_grid = {'n_estimators':[100,200], 'max_depth':[3,5], 'learning_rate':[0.05,0.1]}
grid = GridSearchCV(XGBRegressor(random_state=42), param_grid, cv=tscv,
                    scoring='neg_mean_absolute_error', n_jobs=-1)
grid.fit(X_train, y_train)
best_model = grid.best_estimator_
print('Best params:', grid.best_params_)

## 6. Evaluation (RMSE fixed for sklearn ≥1.3)

In [ ]:
y_pred = best_model.predict(X_test)

print(f"MAE : {mean_absolute_error(y_test, y_pred):.1f} kt")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.1f} kt")
print(f"R²  : {r2_score(y_test, y_pred):.3f}")

results = pd.DataFrame({
    'date': data['date'].iloc[train_size:],
    'actual': y_test.values,
    'predicted': y_pred
}).set_index('date')

plt.plot(results.index, results['actual'],    label='Actual',    marker='o')
plt.plot(results.index, results['predicted'],label='Predicted',marker='s')
plt.title('Actual vs Predicted CO₂')
plt.ylabel('kt'); plt.legend(); plt.xticks(rotation=45); plt.tight_layout(); plt.show()

## 7. Feature Importance

In [ ]:
pd.Series(best_model.feature_importances_, index=feature_cols)\
  .sort_values(ascending=False).head(10)\
  .plot(kind='barh', color='teal')
plt.title('Top 10 Features'); plt.show()

## 8. Save Model & Data

In [ ]:
joblib.dump(best_model, 'xgb_carbon_model.pkl')
data.to_csv('nyc_carbon_data.csv', index=False)
print('Saved: xgb_carbon_model.pkl  &  nyc_carbon_data.csv')